In [44]:
import tkinter as tk
import numpy as np
from tkinter import ttk
from ortools.sat.python import cp_model


In [45]:
import ortools
print(ortools.__version__)


9.6.2534


In [46]:
colors = ['#FDEDEC', '#F7DC6F', '#82E0AA', '#FADBD8', '#D7BDE2', '#FDEBD0', '#A9DF9C', '#EBDEF0', '#F9E79F', '#E8DAEF', '#A3E4D7', '#F5B7B1', '#D2B4DE', '#FCF3CF', '#ABEBC6', '#E6B0AA', '#BB8FCE', '#FAD7A0', '#A2D9CE', '#F5CBA7', '#E6E6E6', '#F2D7D5', '#C39BD3', '#E8DAEF', '#A9DF9C',
          '#E59866', '#A9CCE3', '#D7BDE2', '#F9E79F', '#82E0AA', '#FADBD8', '#D2B4DE', '#A3E4D7', '#FCF3CF', '#EBDEF0', '#F5B7B1', '#ABEBC6', '#E6B0AA', '#BB8FCE', '#FAD7A0', '#A2D9CE', '#F5CBA7', '#E6E6E6', '#F2D7D5', '#C39BD3', '#E8DAEF', '#A9DF9C', '#E59866', '#A9CCE3', '#D7BDE2']
options = ["Opción 1", "Opción 2", "Personalizado"]
operations = ['+', '-', '×', '÷']
matrix = []
count = 0

In [47]:
def draw_matriz(root, matrix):
    for i in range(10):
        for j in range(10):
            label = tk.Label(root, text=matrix[i][j], bg=colors[matrix[i][j]], font=("Arial", 6))
            label.place(x=25+j*25, y=100+i*25, width=25, height=25)
    return

def on_select(combo_boxes):
    selected_values = []
    for combo_box in combo_boxes:
        value=combo_box.get()
        selected_values.append(value)
    return selected_values

def rest(lista):
    if lista:
        resultado = lista[0]
        for i in range(1, len(lista)):
            resultado -= lista[i]
    return resultado

# def mi_funcion(*args):
#     def multiplicacion(a, b):
#         return a * b

#     return reduce(multiplicacion, args)
    
# def div(lista):
#     resultado = None
#     if lista:
#         resultado = lista[0]
#         for i in range(1, len(lista)):
#             if lista[i] != 0:
#                 resultado /= lista[i]
#     return int(resultado)



In [48]:
def insert_results(root,count,matrix):
    label = tk.Label(root, bg="#117A65")
    label.place(x=290, y=90, width=490, height=375)
    xx = 300
    yy = 100
    valores_unicos, recuentos = np.unique(matrix, return_counts=True)
    valores_una_vez = valores_unicos[recuentos == 1]
    entry_vars=[]
    combo_boxes = []

    for i in range(count):
        if i % 16 == 0 and i != 0:
            xx += 160
            yy = 100    
        label = tk.Label(root, text="fig"+str(i)+" =", bg=colors[i])
        label.place(x=xx, y=yy, width=50)
        
        if(i not in valores_una_vez):
            combo_box = ttk.Combobox(root, values = operations)
            combo_box.place(x=xx+50, y=yy, width=50)
            combo_boxes.append(combo_box)
        else:
            label = tk.Label(root,text="=",bg="#000",fg="#fff") 
            label.place(x=xx+50, y=yy, width=50)
        
        var=tk.IntVar()
        entry_vars.append(var)
        entry = tk.Entry(root,textvariable=var)
        entry.place(x=xx+100, y=yy, width=50)
        yy += 20
    
    boton = tk.Button(root, text="Enviar",command=lambda: find_solution(root,entry_vars,valores_una_vez,combo_boxes), width=35)
    boton.place(x=400, y=430)
    return

def find_solution(root,entry_vars,valores_una_vez,combo_boxes):
    operaciones = on_select(combo_boxes)
    global matrix
    global count
    # print(operaciones)
    ken=np.zeros([10,10],dtype=int)
    for i in range(10):
        for j in range(10):
            if matrix[i][j] in valores_una_vez:
                ken[i][j]=entry_vars[matrix[i][j]].get()
                
#----------------Creacion del modelo
    model = cp_model.CpModel()
    board = {}
    for i in range(10):
        for j in range(10):
            if matrix[i][j] not in valores_una_vez:
                board[(i, j)] = model.NewIntVar(1, 10, f'cell_{i}_{j}')
            else:
                board[(i, j)] = ken[i][j]
#---------------Constrains
    for i in range(10):
        model.AddAllDifferent([board[(i, j)] for j in range(10)])
        model.AddAllDifferent([board[(j, i)] for j in range(10)])

    valores_unicos, recuentos = np.unique(matrix, return_counts=True)
    figuras = valores_unicos[recuentos != 1]
    cont=0
    for fig in figuras:
        # -----------
        if(fig==0):
            cont+=1
            continue
        # ----------
        sublist=[]
        for i in range(10):
            for j in range(10):
                if matrix[i][j] == fig:
                    sublist.append(board[(i, j)])
        if operaciones[cont]=='+':
            model.Add(sum(sublist) == entry_vars[fig].get())
        
        elif operaciones[cont]=='-':
            model.Add(rest(sublist) == entry_vars[fig].get())
        
        elif operaciones[cont]=='×':
            model.AddMultiplicationEquality(entry_vars[fig].get(), sublist)
        
        elif operaciones[cont]=='÷':
            model.Add(sublist[0] == sublist[1] * entry_vars[fig])  # Representa la división
        cont+=1
#-----------------Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status ==cp_model.OPTIMAL:
        for i in range(10):
            for j in range(10):
                ken[i][j]=solver.Value(board[(i, j)])
        for i in range(10):
            for j in range(10):
                if matrix[i][j] in valores_una_vez:
                    label = tk.Label(root, text=ken[i][j], bg=colors[matrix[i][j]], font=("Arial", 14,"bold"))
                else:
                    label = tk.Label(root, text=ken[i][j], bg=colors[matrix[i][j]], font=("Arial", 10))
                label.place(x=800+j*40, y=100+i*40, width=40, height=40)
    else:
        print('No se encontró solución')

    


In [49]:
def drow_custom_matrix(custom_matrix,root):
    global matrix
    index=0
    for i in range(10):
        for j in range(10):
            matrix[i][j]=custom_matrix[index].get()
            index+=1
    unique_values = np.unique(matrix)
    count = unique_values.size
    draw_matriz(root, matrix)
    insert_results(root, count,matrix)

def select_matriz(root, option):
    global matrix
    global count
    matrix=np.zeros([10,10],dtype=int)

    if option == "Opción 1":
        matrix = np.array([[0,  1,  2,  2,  2,  3,  3,  4,  5,  5],
                           [0,  1,  7,  2,  2,  3,  3,  4,  4,  11],
                           [6,  6,  7,  7,  8,  9,  10, 4,  11, 11],
                           [12, 13, 13, 13, 13, 9,  9,  14, 14, 15],
                           [16, 17, 18, 19, 19, 19, 20, 14, 21, 22],
                           [17, 17, 23, 24, 24, 19, 25, 26, 21, 22],
                           [27, 28, 23, 24, 29, 30, 25, 31, 31, 32],
                           [27, 33, 33, 34, 35, 30, 30, 36, 36, 32],
                           [27, 33, 33, 35, 35, 30, 36, 36, 37, 37],
                           [38, 39, 39, 40, 41, 42, 42, 42, 42, 42],])
        
    if option == "Opción 2":
        matrix = np.array([[0,  0,  1,  2,  2,  3,  3,  4,  5,  5],
                           [0,  6,  6,  2,  2,  7,  8,  4,  9,  9],
                           [10, 6,  2,  2,  11, 7,  8,  12, 12, 13],
                           [10, 14, 15, 16, 11, 17, 18, 19, 20, 13],
                           [21, 22, 15, 16, 23, 17, 18, 19, 20, 24],
                           [25, 26, 26, 27, 27, 28, 29, 29, 30, 30],
                           [31, 26, 32, 32, 32, 28, 33, 33, 34, 35],
                           [31, 36, 32, 32, 37, 33, 33, 38, 34, 39],
                           [40, 36, 41, 41, 37, 33, 33, 38, 42, 39],
                           [40, 43, 43, 41, 44, 33, 45, 45, 46, 46],])
    if option == "Personalizado":
        entry_vars=[]
        for i in range(10):
            for j in range(10):
                var=tk.IntVar()
                entry_vars.append(var)
                entry = tk.Entry(root,textvariable=var)
                entry.place(x=25+j*25, y=100+i*25, width=25, height=25)
        
        boton_2 = tk.Button(root, text="Enviar Matriz Personalizada",command=lambda: drow_custom_matrix(entry_vars,root), width=35)
        boton_2.place(x=25, y=400)

        return 
    
    unique_values = np.unique(matrix)
    count = unique_values.size
    draw_matriz(root, matrix)
    insert_results(root, count,matrix)
    return

In [50]:
root = tk.Tk()
root.geometry("1225x525")
root.configure(bg="#48C9B0")

label_1 = tk.Label(root, text="KEN KEN", font=("Arial", 18, "bold"), bg="#117A65", fg="#FFF")
label_1.pack(fill=tk.X)
# Tablero-------------------------
label_2 = tk.Label(root, text="Tablero", font=("Arial", 14, "bold"), bg="#48C9B0")
label_2.place(x=25, y=50)
label_3 = tk.Label(root, bg="#117A65")
label_3.place(x=25, y=100, width=250, height=250)
label_4 = tk.Label(root, text="Elegir plantilla:", bg="#48C9B0")
label_4.place(x=25, y=375)

selected_option = tk.StringVar()
combo_box_1 = ttk.Combobox(root, textvariable=selected_option, values=options)
combo_box_1.place(x=125, y=375)
combo_box_1.current(0)
boton_1 = tk.Button(root, text="Enviar", command=lambda: select_matriz(root, combo_box_1.get()), width=35)
boton_1.place(x=25, y=430)

# Ingresar resultados-------------
label_2 = tk.Label(root, text="Resultados", font=("Arial", 14, "bold"), bg="#48C9B0")
label_2.place(x=300, y=50)
label_4 = tk.Label(root, bg="#117A65")
label_4.place(x=290, y=90, width=490, height=375)

# Ingresar solution-------------
label_2 = tk.Label(root, text="Solución", font=("Arial", 14, "bold"), bg="#48C9B0")
label_2.place(x=800, y=50)
label_4 = tk.Label(root, bg="#117A65")
label_4.place(x=800, y=100, width=400, height=400)

root.mainloop()